## A Markov Chain for Credit Ratings

In mathematical finance, Markov chains are used to model the default risk of a company or country (more specifically, the default of a company's or country's liability like a corporate or government bond - see e.g. PSTAT 171 and PSTAT 170). 

Rating agencies (like <i>Standard & Poor’s</i>, <i>Moody’s</i>, <i>Fitch</i>) rate the financial stability of a company and classify them according to different classes. A possible classification may range from 'AAA' for debitors with a very good credit rating to 'CCC' for debitors which are very likely to delay in paying a debt; and 'D' for those debitors which can't satisfy their financial liabilities anymore (in other words, they are default). 

The yearly credit rating of a company can be modeled as a Markov chain $(X_n)_{n=0,1,2,\ldots}$ with state space

$$ \mathcal{S} = \{ AAA,\, AA,\, A,\, BBB,\, BB,\, B,\, CCC,\, D \} $$

where $X_n$ represents the credit rating class of a company in the $n$-th year.

In [1]:
import scipy.linalg
import numpy as np

In [2]:
P = np.array([[92.07, 7.09, 0.63, 0.15, 0.06, 0.00, 0.00, 0.00],\
            [0.62, 90.84, 7.76, 0.59, 0.06, 0.10, 0.02, 0.01],\
            [0.05, 2.09, 91.38, 5.79, 0.44, 0.16, 0.04, 0.05],\
            [0.03, 0.21, 4.10, 89.37, 4.82, 0.86, 0.24, 0.37],\
            [0.03, 0.08, 0.40, 5.54, 83.24, 8.15, 1.11, 1.45],\
            [0.00, 0.08, 0.27, 0.34, 5.39, 82.41, 4.92, 6.59],\
            [0.10, 0.00, 0.29, 0.58, 1.55, 10.54, 52.80, 34.14],\
            [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 100.0]])

P /= 100

P

array([[9.207e-01, 7.090e-02, 6.300e-03, 1.500e-03, 6.000e-04, 0.000e+00,
        0.000e+00, 0.000e+00],
       [6.200e-03, 9.084e-01, 7.760e-02, 5.900e-03, 6.000e-04, 1.000e-03,
        2.000e-04, 1.000e-04],
       [5.000e-04, 2.090e-02, 9.138e-01, 5.790e-02, 4.400e-03, 1.600e-03,
        4.000e-04, 5.000e-04],
       [3.000e-04, 2.100e-03, 4.100e-02, 8.937e-01, 4.820e-02, 8.600e-03,
        2.400e-03, 3.700e-03],
       [3.000e-04, 8.000e-04, 4.000e-03, 5.540e-02, 8.324e-01, 8.150e-02,
        1.110e-02, 1.450e-02],
       [0.000e+00, 8.000e-04, 2.700e-03, 3.400e-03, 5.390e-02, 8.241e-01,
        4.920e-02, 6.590e-02],
       [1.000e-03, 0.000e+00, 2.900e-03, 5.800e-03, 1.550e-02, 1.054e-01,
        5.280e-01, 3.414e-01],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00]])

As warm-up let us <u>compute the probability</u> that a company which is rated with 'AAA' today will <u>not</u> default during the next 8 years.

In [3]:
1 - np.linalg.matrix_power(P, 8)[0, 7]

0.9982299711480223

So there is only 0.2% chance of a default. But for a company that is BBB rated default is much likelier:

In [4]:
np.linalg.matrix_power(P, 8)[3, 7]

0.06957603261513783

## Finding the long-term weather distribution 

Recall the weather Markov chain representing Rainy, Sunny and Cloudy days:

In [5]:
weatherP = np.array([[0.2,0.6,0.2],\
                    [0.1,0.8,0.1],\
                    [0.1,0.6,0.3]])

weatherP

array([[0.2, 0.6, 0.2],
       [0.1, 0.8, 0.1],
       [0.1, 0.6, 0.3]])

We can see the convergence if we say look at $P^5, P^{10}, P^{20}$:

In [6]:
print( np.linalg.matrix_power( weatherP, 5 ))
print()
print( np.linalg.matrix_power( weatherP, 10 ))
print()
print( np.linalg.matrix_power( weatherP, 20 ))

[[0.11112 0.74976 0.13912]
 [0.11111 0.75008 0.13881]
 [0.11111 0.74976 0.13913]]

[[0.11111111 0.74999992 0.13888897]
 [0.11111111 0.75000003 0.13888886]
 [0.11111111 0.74999992 0.13888897]]

[[0.11111111 0.75       0.13888889]
 [0.11111111 0.75       0.13888889]
 [0.11111111 0.75       0.13888889]]


To compute the stationary distribution $\pi$ that solves $\pi P = \pi$ in Python we use the eigenvector decomposition of the matrix $P$. 

In [7]:
eigenvalues, eigenvector = scipy.linalg.eig(weatherP.T)

In [8]:
print(eigenvalues)

[1. +0.j 0.1+0.j 0.2+0.j]


In [9]:
print(eigenvector)

[[-1.44149994e-01 -7.07106781e-01  1.79439138e-16]
 [-9.73012460e-01  2.16156920e-16 -7.07106781e-01]
 [-1.80187493e-01  7.07106781e-01  7.07106781e-01]]


Namely, $\pi$ is the normalized first eigenvector of $P$. To normalize means to make sure that it is a probability vector, i.e. adds up to 1.

In [10]:
pi = eigenvector[:, 0]

In [11]:
print(pi)

[-0.14414999 -0.97301246 -0.18018749]


In [12]:
np.sum(pi)

-1.297349946281605

In [13]:
scipy.linalg.norm(pi, 2)

1.0

In [14]:
newPi = eigenvector[:,0] / np.sum(pi)

In [15]:
print(newPi)

[0.11111111 0.75       0.13888889]


In [16]:
4/36

0.1111111111111111

In [17]:
27/36

0.75

In [18]:
5/36

0.1388888888888889

So algebraically, $\pi = [1/9 \quad 3/4 \quad 5/36]$, hence it is Sunny for 3/4 of all days in the long term.

### Long-term credit rating?

We can repeat the same computations (which only take 2 lines) for the 7-state Credit Migration matrix P. Here is what we get:

In [19]:
eigenvalP, eigenvecP = scipy.linalg.eig(P.T)

longtermCredit = eigenvecP[:,0] / np.sum(eigenvecP[:, 0])

print(longtermCredit)

[0. 0. 0. 0. 0. 0. 0. 1.]


So default is inevitable!